<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/annotation_avec_LLM/gpt_5_1_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# =========================
# 0. CLIENT (GPT-4.1)
# =========================

client = OpenAI(api_key="")

# =========================
# 1. MODÈLE
# =========================

MODEL_NAME = "gpt-5.1"

# =========================
# 2. BASELINE PROMPT
# =========================

BASELINE_PROMPT = """
Tu es linguiste.

Ta tâche est de classer une expression nominale française
dans UNE SEULE des catégories suivantes, en te fondant sur
une compréhension linguistique générale (pas de guide spécifique).

Catégories :

1) Expression_idiomatique
Expression figée dont le sens global n’est pas directement déductible
du sens littéral de ses mots.
Exemples : « lune de miel », « forêt noire » (gâteau)

2) Collocation_opaque
Association de mots relativement conventionnelle,
dont le sens implique une image, une métaphore ou une métonymie.
Exemples : « fil rouge », « train de vie »

3) Collocation_transparente
Association de mots fréquente ou conventionnelle,
dont le sens est globalement déductible des mots qui la composent.
Exemples : « événement culturel », « roman policier »

4) Expression_libre
Combinaison de mots construite librement en discours,
sans caractère figé ou conventionnel particulier.
Exemples : « livre intéressant », « maison ancienne »

Contraintes :
- Choisis UNE seule catégorie.
- Réponds uniquement selon le format ci-dessous.

FORMAT OBLIGATOIRE :
Catégorie : <Expression_idiomatique | Collocation_opaque | Collocation_transparente | Expression_libre>
Explication : <une phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

ALLOWED = {
    "Expression_idiomatique",
    "Collocation_opaque",
    "Collocation_transparente",
    "Expression_libre",
}

# =========================
# 3. APPEL LLM (baseline)
# =========================

def classify_expression_baseline(expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)

    prompt = BASELINE_PROMPT.format(
        expression=str(expression).strip(),
        contexte=contexte
    )

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=80,
    )

    text = (resp.choices[0].message.content or "").strip()

    # validation minimale du format/catégorie
    cat = None
    for line in text.splitlines():
        if line.lower().startswith("catégorie"):
            cat = line.split(":", 1)[-1].strip()
            break

    if cat not in ALLOWED:
        text = (
            "Catégorie : INVALID\n"
            "Explication : Réponse hors format.\n\n"
            + text
        )

    return text

# =========================
# 4. TRAITEMENT EXCEL
# =========================

input_file = "nominal_part_7.xlsx"
df = pd.read_excel(input_file)
df["llm_raw_response"] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Baseline GPT-4.1 annotation"):
    expr = row.get("expression")
    ex = row.get("examples_joined")

    if pd.isna(expr) or not str(expr).strip():
        df.at[idx, "llm_raw_response"] = "N/A"
        continue

    df.at[idx, "llm_raw_response"] = classify_expression_baseline(expr, ex)

output_file = "annotated_nominal_part_7_gpt41_baseline_general_defs.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved: {output_file}")
